In [11]:
import pandas as pd
import sqlite3
import numpy as np
import munkres



In [12]:
%config IPCompleter.greedy=True

In [13]:

conn = sqlite3.connect('D:\CSGOProGames\CSPAdb.sqlite3')
match_links = pd.read_sql_query("""SELECT * FROM analyzer_MatchInfo""", conn)

playerdata = pd.read_sql_query("""SELECT time, team, XPos, YPos, weapon, game_id, t_score, ct_score, win_reason, match_id
                            FROM analyzer_PlayerData
                            JOIN analyzer_GameInfo 
                            ON analyzer_PlayerData.game_id = analyzer_GameInfo.id
                            WHERE played_map = 'inferno'
                            LIMIT 150000""", conn)

# Nades have different time stamps
nadedata = pd.read_sql_query("""SELECT time, XPos, YPos, nade_type, game_id
                        FROM analyzer_Nades
                        JOIN analyzer_GameInfo 
                        ON analyzer_Nades.game_id = analyzer_GameInfo.id
                        WHERE played_map = 'inferno'
                        LIMIT 150000""", conn)
conn.close()

In [14]:
fixNames = playerdata['team'].replace('Terrorist', 'T').replace('CounterTerrorist', 'CT')

In [15]:
playerdata['team'] = fixNames

In [16]:
nadedata.head()

,time,XPos,YPos,nade_type,game_id
0,139.6,1192.6490,462.38760,HE,217
1,139.9,748.4474,-125.77450,Flash,217
2,141.6,786.7012,-83.28719,Flash,217
3,142.1,400.3172,2059.36300,Decoy,217
4,145.6,862.3829,-209.94330,Decoy,217


In [17]:
bytick = playerdata.groupby(['game_id','time'])

In [23]:
for tick in bytick:
    tick = tick[1]  # Tick gives a tuple of ((time, game), dataframe)

        
    win_reason = tick.iloc[0]['win_reason']
    t_score = tick.iloc[0]['t_score']
    ct_score = tick.iloc[0]['ct_score']
    hltv_link = match_links.loc[(match_links.id == tick.iloc[0]['match_id'])]['hltv_link'].values[0]
    time = tick.iloc[0]['time']
    print(type(int(t_score)))
    print(ct_score)
    print(hltv_link)
    print(type(time))
    break

<class 'int'>
0
/matches/2313257/opaa-vs-invictus-aquilas-esl-major-league-summer-2017
<class 'numpy.float64'>


In [20]:
bytick.head(5)

,id,time,team,playernum,XPos,YPos,ZPos,weapon,game_id,id,played_map,t_score,ct_score,win_reason,match_id
0,376845,0.0,T,4,-1506.850,671.6946,-43.96875,Knife,217,217,inferno,0,0,TerroristWin,6
1,376846,0.0,T,5,-1655.827,418.4227,-63.34569,Knife,217,217,inferno,0,0,TerroristWin,6
2,376847,0.0,CT,6,2292.060,2027.6900,128.03130,Knife,217,217,inferno,0,0,TerroristWin,6
3,376848,0.0,CT,7,2363.130,1985.6600,128.03130,Knife,217,217,inferno,0,0,TerroristWin,6
4,376849,0.0,T,8,-1661.562,290.4704,-63.23479,Knife,217,217,inferno,0,0,TerroristWin,6
9,376854,0.5,T,3,-1652.105,594.5435,-56.96875,Knife,217,217,inferno,0,0,TerroristWin,6
10,376855,0.5,T,4,-1495.010,574.9493,-61.27203,Knife,217,217,inferno,0,0,TerroristWin,6
11,376856,0.5,T,5,-1583.178,373.6241,-66.33540,Knife,217,217,inferno,0,0,TerroristWin,6
12,376857,0.5,CT,6,2292.060,2027.6900,128.03130,Knife,217,217,inferno,0,0,TerroristWin,6
13,376858,0.5,CT,7,2363.130,1985.6600,128.03130,Knife,217,217,inferno,0,0,TerroristWin,6


In [26]:
for tick in bytick:
    print(tick)
    break

((217, 0.0),        id  time team  playernum      XPos       YPos       ZPos weapon  \
0  376845   0.0    T          4 -1506.850   671.6946  -43.96875  Knife   
1  376846   0.0    T          5 -1655.827   418.4227  -63.34569  Knife   
2  376847   0.0   CT          6  2292.060  2027.6900  128.03130  Knife   
3  376848   0.0   CT          7  2363.130  1985.6600  128.03130  Knife   
4  376849   0.0    T          8 -1661.562   290.4704  -63.23479  Knife   
5  376850   0.0   CT          9  2374.470  2171.8500  128.03130  Knife   
6  376851   0.0   CT         10  2433.620  2157.4100  128.03130  Knife   
7  376852   0.0   CT         11  2449.140  2010.2200  128.03130  Knife   
8  376853   0.0    T         12 -1491.000   490.0000  -63.96875  Knife   

   game_id   id played_map  t_score  ct_score    win_reason  match_id  
0      217  217    inferno        0         0  TerroristWin         6  
1      217  217    inferno        0         0  TerroristWin         6  
2      217  217    inferno    

In [27]:
players = [['CT', 'Dead'],['T', 'Any', 112, 114],['T', 'Dead']]
nades = []
threshold = 0.85
distancescale = 150.0
xoffset = 1760.0
yoffset = 790.0


matchinggames = []
matches = set()


for tick in bytick:
    tick = tick[1]  # Tick gives a tuple of ((time, game), dataframe)
    # Already included this game.
    if tick.iloc[0]['game_id'] in matches:
        continue
    
    nadesimilarity = 0
    playersimilarity = 0
    
    if players:
        similarities = []
        # We need to go through the entire tick before we know how many living players there are
        dead_T = 5
        dead_CT = 5
        wanted_dead_T = 0
        wanted_dead_CT = 0
        for i,playerrow in tick.iterrows():
            tempscore = []

            if playerrow['team'] == 'T':
                dead_T -= 1
            elif playerrow['team'] == 'CT':
                dead_CT -= 1

            for player in players:
                score = 0.0          
                if player[0] == 'T' and player[1] == 'Dead':
                    tempscore.append('DT')
                    wanted_dead_T += 1
                    continue
                elif player[0] == 'CT' and player[1] == 'Dead':
                    tempscore.append('DCT')
                    wanted_dead_CT += 1
                    continue

                if playerrow['team'] == player[0]:
                    if player[1] == playerrow['weapon'] or player[1] == 'Any':
                        score += 1
                    distance = math.sqrt(((playerrow['XPos']+xoffset)-(player[2]*9.75))**2 + ((playerrow['YPos']+yoffset)-(player[3]*9.75))**2)
                    score += 2**-(distance/distancescale)
                    tempscore.append(score)
                else:
                    tempscore.append(0)
            similarities.append(tempscore)

        # This seems ot make the code slower somehow? ~40 s vs ~60 s
        #Skip ahead because this tick is very unlikely to have a high similarity score
    #         if wanted_dead_T/len(tick.index) > dead_T or wanted_dead_CT/len(tick.index) > dead_CT:
    #             continue                  

        colmaxes = []        
        for i,row in enumerate(similarities):
            for j,column in enumerate(row):                 
                if column == 'DCT':
                    if dead_CT > 0:
                        similarities[i][j] = 2 # Determine good value
                    else:
                        similarities[i][j] = 0
                elif column == 'DT':
                    if dead_T > 0:
                        similarities[i][j] = 2 
                    else:
                        similarities[i][j] = 0
                try:
                    if colmaxes[j] < similarities[i][j]:
                        colmaxes[j] = similarities[i][j]
                except:
                    colmaxes.append(0)
                # Turn profit function into cost function        
                similarities[i][j] = 2.0 - similarities[i][j]


        if sum(colmaxes)/(len(players)*2) < (threshold): # Have to at least have the possibility of exceeding the threshold
            continue

        # Assignment problem : https://en.wikipedia.org/wiki/Assignment_problem
        # Solved in O(n^3) using this algorithm
        # http://software.clapper.org/munkres/
        # This algorithm uses a cost calculation (i.e., minimization)
        # so we first need to subtract the values from the max value 
        # (performed during dead player value assignment)

        playersimilarity = 0
        if similarities:
            m = munkres.Munkres()
            indices = m.compute(similarities)
            for row,column in indices:
                # Revert to a similarity (high is better) and calculat total
                playersimilarity += 2.0 - similarities[row][column]


    if nades:
        similarities = []
        nadesimilarity = 0
        # Get all the nades from the same game within a defined time range.
        naderange = 2 
        naderows = nadedata.loc[(nadedata.time < playerrow.time + naderange) & (nadedata.time > playerrow.time - naderange) & (nadedata.game_id == playerrow.game_id)]
        if len(naderows.index) >= len(nades):
            for i,naderow in naderows.iterrows():
                tempscore = []
                for nade in nades:
                    score = 0.0       
                    if naderow['nade_type'] == nade[0]: 
                        # Customized for each map
                        distance = math.sqrt(((naderow['XPos']+xoffset)-(nade[1]*9.75))**2 + ((naderow['YPos']+yoffset)-(nade[2]*9.75))**2)
                        # Will need some kind of normalization
                        score += 2**-(distance/distancescale)
                        tempscore.append(score)
                    else:
                        tempscore.append(0)

                similarities.append(tempscore)

            for i,row in enumerate(similarities):
                for j,column in enumerate(row):
                    # Turn profit function into cost function
                    similarities[i][j] = 1 - similarities[i][j]

            if similarities:
                m = munkres.Munkres()
                indices = m.compute(similarities)
                for row,column in indices:        
                    # Revert to a similarity (high is better) and calculate total
                    nadesimilarity += 1 - similarities[row][column] 
        
    totalsimilarity = (float(playersimilarity) + float(nadesimilarity))/((2*len(players))+len(nades))  # Weighted normalization      

    if totalsimilarity > threshold:
        print(totalsimilarity, playersimilarity, nadesimilarity)
        matchinggames.append([totalsimilarity, playerrow['win_reason'], playerrow['match_id'], playerrow['t_score'], playerrow['ct_score']])  # Use match id to get match link later
        matches.add(playerrow['game_id'])
            
print(matchinggames)
                

0.9397222617231563 5.638333570338938 0
0.9767098084406604 5.860258850643962 0
0.8585845463230538 5.151507277938323 0
0.8579249783676887 5.147549870206132 0
0.853413351619578 5.120480109717468 0
0.8569377236626554 5.1416263419759325 0
0.8671341669516491 5.202805001709895 0
0.8534926053310504 5.120955631986303 0
0.856661215345444 5.139967292072664 0
0.8716166733931647 5.229700040358988 0
0.850876243548457 5.105257461290742 0
[[0.9397222617231563, 'TerroristWin', 6, 0, 0], [0.9767098084406604, 'CTWin', 6, 1, 3], [0.8585845463230538, 'TerroristWin', 6, 3, 5], [0.8579249783676887, 'TerroristWin', 6, 4, 5], [0.853413351619578, 'TargetBombed', 6, 12, 12], [0.8569377236626554, 'CTWin', 7, 0, 0], [0.8671341669516491, 'TerroristWin', 7, 0, 3], [0.8534926053310504, 'TargetBombed', 8, 1, 3], [0.856661215345444, 'TargetBombed', 8, 2, 3], [0.8716166733931647, 'TerroristWin', 8, 4, 5], [0.850876243548457, 'Draw', 8, 6, 5]]


In [28]:
len(matchinggames)

11